In [1]:
%load_ext lab_black

In [24]:
import json
import gzip
import pandas as pd
import glob
import tempfile
import uuid
import os
from pprint import pprint
from multiprocessing import Pool
from pathlib import Path
from sadie.airr import Airr, AirrTable

pd.set_option("display.max_rows", 45)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 100000)

## Try and find OAS similariities

I'm suspect of the entire airr file since it doesn't have fwr4

In [4]:
df_bz2 = pd.read_csv(
    "https://sadie.s3.us-east-2.amazonaws.com/integration/OAS_sample_subsample.bz2",
    index_col=0,
).reset_index()

/Users/jordanwillis/miniconda3/envs/sadie/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (27,43,44,45,46,71,105) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [39]:
with open("../../tests/integration/airr/fixtures/OAS_subsample.fasta", "w") as f:
    for index, (i, k, j) in enumerate(
        zip(
            df_bz2["OAS_dataunit"].str.split(".csv.gz").str.get(0),
            df_bz2["OAS_dataindex"],
            df_bz2["sequence"],
        )
    ):
        f.write(f">{index}_{i}_{k}\n{j}\n")

In [40]:
i, k, j

('SRR11937598_igblastn_anarci_Light_Bulk',
 120,
 'GATGGTGCAGCCACAGCTCTGGCACCAGGGGTCCCTTCCAATATCAGCACCATGGCCTGGACTCCTCTCTTTCTGTTCCTCCTCACTTGCTGCCCAGGGTCCAATTCCCAGGCTTTTGTGACTCAGGAGACCTCACTGACTGTGTNNCNNNNNNGGACAGTCACTCTCACCTGTGGCTCCAGCACTGGAGCTGTCACCAGTGGTCATTTTCCCTACTGGTTCCAGCAGAAGCCTGGCCAAGCCCCCAGGACACTGATTTATGATACAAGCAACAAACACTCCTGGACACCTGCCCGTTTCTCAGGCTCCCTCCTTGGGGGCAAAGCTGCCCTGACCCTTTCGGGTGCGCAGCCTGAGGATGAGGCTGAGTATTACTGCTTACTAACCTACATTGGCGCCATGGTATTCGGCGGAGGGACCAAGCTGACCGTCCTAGGTCAGCCCAAGGCTGCCCCCTCGGTCACTCTGTTCCCAC')

In [31]:
print([i for i in list(df_bz2.columns) if i.startswith("fwr")])

['fwr1', 'fwr1_aa', 'fwr2', 'fwr2_aa', 'fwr3', 'fwr3_aa', 'fwr1_start', 'fwr1_end', 'fwr2_start', 'fwr2_end', 'fwr3_start', 'fwr3_end']


In [5]:
airr_api = Airr(species="human", database="imgt", functional="all")

In [6]:
airr_dataframe = airr_api.run_dataframe(df_bz2, "index", "sequence")

/Users/jordanwillis/repos/personal/sadie/src/sadie/airr/airr.py:410: DtypeWarning: Columns (10,26,27,28,29,56,92) have mixed types.Specify dtype option on import or set low_memory=False.
  return self.run_multiple(_get_seq_generator(), scfv=scfv)
Caution - sequences [5664, 9858, 19830] have FW1-FW3 resolved but do not have the CDR3,
                that is indicative that the J gene penalty needs to be lowered. Or that
                air_api.igblast.j_penalty = -1
             


In [7]:
ref = df_bz2[df_bz2.columns.intersection(airr_dataframe.table.columns)].drop(
    "sequence_id", axis=1
)
target = airr_dataframe.table[
    airr_dataframe.table.columns.intersection(df_bz2.columns)
].drop("sequence_id", axis=1)

In [32]:
ref.re

,sequence,locus,stop_codon,vj_in_frame,productive,rev_comp,v_call,d_call,j_call,sequence_alignment,germline_alignment,sequence_alignment_aa,germline_alignment_aa,v_alignment_start,v_alignment_end,d_alignment_start,d_alignment_end,j_alignment_start,j_alignment_end,v_sequence_alignment,v_sequence_alignment_aa,v_germline_alignment,v_germline_alignment_aa,d_sequence_alignment,d_sequence_alignment_aa,d_germline_alignment,d_germline_alignment_aa,j_sequence_alignment,j_sequence_alignment_aa,j_germline_alignment,j_germline_alignment_aa,fwr1,fwr1_aa,cdr1,cdr1_aa,fwr2,fwr2_aa,cdr2,cdr2_aa,fwr3,fwr3_aa,cdr3,cdr3_aa,junction,junction_length,junction_aa,junction_aa_length,v_score,d_score,j_score,v_cigar,d_cigar,j_cigar,v_support,d_support,j_support,v_identity,d_identity,j_identity,v_sequence_start,v_sequence_end,v_germline_start,v_germline_end,d_sequence_start,d_sequence_end,d_germline_start,d_germline_end,j_sequence_start,j_sequence_end,j_germline_start,j_germline_end,fwr1_start,fwr1_end,cdr1_start,cdr1_end,fwr2_start,fwr2_end,cdr2_start,cdr2_end,fwr3_start,fwr3_end,cdr3_start,cdr3_end,np1,np1_length,np2,np2_length
0,ACTGTGTCTGGTGGTCTCATCACTAGTGATGGTCATTACTGGAGTT...,IGH,F,T,T,F,IGHV4-31*02,IGHD1-1*01,IGHJ3*02,ACTGTGTCTGGTGGTCTCATCACTAGTGATGGTCATTACTGGAGTT...,ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCT...,TVSGGLITSDGHYWSWIRQSPGKGLEWLGSTYYNGATYYSESLESR...,TVSGGSISSGGYYWSWIRQHPGKGLEWIGYIYYSGSTYYNPSLKSR...,1.0,229.0,238.0,242.0,249.0,297.0,ACTGTGTCTGGTGGTCTCATCACTAGTGATGGTCATTACTGGAGTT...,TVSGGLITSDGHYWSWIRQSPGKGLEWLGSTYYNGATYYSESLESR...,ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCT...,TVSGGSISSGGYYWSWIRQHPGKGLEWIGYIYYSGSTYYNPSLKSR...,CAACT,QL,CAACT,QL,TGATGCTTTTGATATGTGGGGCCAAGGGACATTGGTCACCGTCTCCTCA,DAFDMWGQGTLVTVSS,TGATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCA,DAFDIWGQGTMVTVSS,ACTGTGTCT,TVS,GGTGGTCTCATCACTAGTGATGGTCATTAC,GGLITSDGHY,TGGAGTTGGATTCGCCAGTCTCCAGGGAAGGGCCTGGAGTGGTTGG...,WSWIRQSPGKGLEWLGS,ACTTATTACAATGGGGCCACC,TYYNGAT,TACTACAGCGAGTCCCTCGAGAGTCGACTCCTCATTTCAATAGACC...,YYSESLESRLLISIDPSQTQFSLKLTSVTAADTAVYYC,GCGACCTGGACCCAACTCCAACTTGATGCTTTTGATATG,ATWTQLQLDAFDM,TGTGCGACCTGGACCCAACTCCAACTTGATGCTTTTGATATGTGG,45.0,CATWTQLQLDAFDMW,15.0,233.995,10.301,77.595,66N229M68S4N,237S4N5M55S8N,248S49M1N,1.170000e-63,103.8000,1.285000e-18,82.533,100.000,93.878,1.0,229.0,67.0,295.0,238.0,242.0,5.0,9.0,249.0,297.0,1.0,49.0,1.0,9.0,10.0,39.0,40.0,90.0,91.0,111.0,112.0,225.0,226.0,264.0,CCTGGACC,8.0,CCAACT,6.0
1,GGCCTCAGTGAAGGTCTCCTGCAAGGCATCTGGATACACCTTCACC...,IGH,F,T,T,T,IGHV1-46*01,IGHD2-2*01,IGHJ4*02,GGCCTCAGTGAAGGTCTCCTGCAAGGCATCTGGATACACCTTCACC...,GGCCTCAGTGAAGGTTTCCTGCAAGGCATCTGGATACACCTTCACC...,ASVKVSCKASGYTFTNHYIHWVRQAPGQGLEWMGIINPSGGRTSHV...,ASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPSGGSTSYA...,1.0,250.0,257.0,278.0,287.0,329.0,GGCCTCAGTGAAGGTCTCCTGCAAGGCATCTGGATACACCTTCACC...,ASVKVSCKASGYTFTNHYIHWVRQAPGQGLEWMGIINPSGGRTSHV...,GGCCTCAGTGAAGGTTTCCTGCAAGGCATCTGGATACACCTTCACC...,ASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPSGGSTSYA...,TGTAGTGCTTCCAACTGCTATG,CSASNCY,TGTAGTAGTACCAGCTGCTATG,CSSTSCY,TTTGACCACTGGGGCCAGGGAACCCTGGTCACCGTCGCCTCAG,FDHWGQGTLVTVAS,TTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,FDYWGQGTLVTVSS,GGCCTCAGTGAAGGTCTCCTGCAAGGCATCT,ASVKVSCKAS,GGATACACCTTCACCAACCACTAC,GYTFTNHY,ATCCACTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGG...,IHWVRQAPGQGLEWMGI,ATCAACCCTAGTGGTGGTAGGACA,INPSGGRT,AGTCACGTACAGGAGTTCCAGGGCAGAGTCACCATGACCAGGGACA...,SHVQEFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYFC,GCGAGACAACAGTGTAGTGCTTCCAACTGCTATGAGGAGAGTTTTG...,ARQQCSASNCYEESFDH,TGTGCGAGACAACAGTGTAGTGCTTCCAACTGCTATGAGGAGAGTT...,57.0,CARQQCSASNCYEESFDHW,19.0,338.388,19.914,71.827,44N250M105S2N,256S7N22M77S2N,286S5N43M26S,5.297000e-95,0.1592,8.419000e-17,93.200,81.818,95.349,1.0,250.0,45.0,294.0,257.0,278.0,8.0,29.0,287.0,329.0,6.0,48.0,1.0,31.0,32.0,55.0,56.0,106.0,107.0,130.0,131.0,244.0,245.0,295.0,CAACAG,6.0,AGGAGAGT,8.0
2,CGGGGAGTCTCTGAAGATCACCTGTGCAGCCTCTGGATTCACCTTC...,IGH,F,T,T,T,IGHV3-74*01,IGHD1-26*01,IGHJ4*02,GGGGAGTCTCTGAAGATCACCTGTGCAGCCTCTGGATTCACCTTCA...,GGGGGGTCCCTGAGACTCT

In [23]:
for index in ref.index:
    sub_ref = ref.loc[index]
    sub_target = target.loc[index]
    aggree = sub_ref[sub_ref == sub_target].index
    disagree = sub_ref[sub_ref != sub_target].index
    if disagree.empty:
        print('yay')
    for dis_index in disagree:
        print(f"OAR-{dis_index}:{sub_ref[dis_index]}")
        print(f"Sadie-{dis_index}:{sub_target[dis_index]}\n")

    break

OAR-stop_codon:F
Sadie-stop_codon:False

OAR-vj_in_frame:T
Sadie-vj_in_frame:True

OAR-productive:T
Sadie-productive:True

OAR-rev_comp:F
Sadie-rev_comp:False

OAR-v_call:IGHV4-31*02
Sadie-v_call:IGHV4-31*02,IGHV4-31*03

OAR-d_call:IGHD1-1*01
Sadie-d_call:IGHD4-23*01

OAR-germline_alignment:ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCTGGATCCGCCAGCACCCAGGGAAGGGCCTGGAGTGGATTGGGTACATCTATTACAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTTACCATATCAGTAGACACGTCTAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACTGCCGCGGACACGGCCGTGTATTACTGTGCGANNNNNNNNCAACTNNNNNNTGATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCA
Sadie-germline_alignment:ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCTGGATCCGCCAGCACCCAGGGAAGGGCCTGGAGTGGATTGGGTACATCTATTACAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTTACCATATCAGTAGACACGTCTAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACTGCCGCGGACACGGCCGTGTATTACTGTGCGANNNNNNNNNAACTCCNNNNTGATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCA

OAR-germline_alignment_aa:TVSGGSISSGGYYWSWIRQHPGKGLEWIGYIYYSGSTYYNPSLKSRVTISVDTSKNQFSL

In [19]:
disagree

Index(['stop_codon', 'vj_in_frame', 'productive', 'rev_comp', 'v_call', 'd_call', 'germline_alignment', 'germline_alignment_aa', 'd_alignment_start', 'd_alignment_end', 'd_sequence_alignment', 'd_sequence_alignment_aa', 'd_germline_alignment', 'd_germline_alignment_aa', 'v_score', 'd_score', 'j_score', 'd_cigar', 'v_support', 'd_support', 'j_support', 'v_identity', 'j_identity', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'np1', 'np1_length', 'np2', 'np2_length'], dtype='object')

In [9]:
from pandas.testing import assert_series_equal

# Get IMGT airr

In [46]:
header = 

ParserError: Error tokenizing data. C error: Expected 125 fields in line 57, saw 128
